### 모두를 위한 머신러닝/딥러닝 강의
김성훈 교수님의 모두를 위한 머신러닝/딥러닝 강의 중 lab 강의 코드입니다.
## Lab9_2 Tensorboard (Neural Net for XOR)
tf.name_scope와 tf.variable_scope의 차이는 아래의 링크를 참고, 서로 다른 parameter를 주고 cost/loss의 변화 같은 것을 체크하고 싶을 때, 동일 폴더안의 세부 폴더에 각각의 graph를 저장하고, 세부 폴더의 상위폴더를 cmd에서 tensorboard 명령어로 실행시킨다.

http://stackoverflow.com/questions/35919020/whats-the-difference-of-name-scope-and-a-variable-scope-in-tensorflow/35927880

* Tensorboard from tf.name_scope : tensorboard --logdir=./logs/xor_with_name_scope
* Tensorboard from tf.variable_scope : tensorboard --logdir=./logs/xor_with_variable_scope
* All tensorboard : tensorboard --logdir=./logs

### with tf.name_scope

In [1]:
import tensorflow as tf
import numpy as np
import os, sys
os.listdir()

['.ipynb_checkpoints',
 'data-01-test-score.csv',
 'data-02-test-score.csv',
 'data-03-test-score.csv',
 'data-04-test-score.csv',
 'data-05-test-score.csv',
 'diabetes.csv',
 'DLEL.zip',
 'DLEL_1_TensorFlow_Basics.ipynb',
 'DLEL_2_Linear_regression.ipynb',
 'DLEL_3_Minimizing_Cost.ipynb',
 'DLEL_4_1_Multi-variable_linear_regression.ipynb',
 'DLEL_4_2_Loading_Data_from_File.ipynb',
 'DLEL_5_Logistic_regression.ipynb',
 'DLEL_6_Softmax_Classifier.ipynb',
 'DLEL_7_1_Learning_rate_&_Evaluation_&_pre-processing.ipynb',
 'DLEL_7_2_Softmax_classifier_for_mnist_data.ipynb',
 'DLEL_8_Tensor_Manipulation.ipynb',
 'DLEL_9_1_NN_for_XOR.ipynb',
 'DLEL_9_2_Tensorboard_(NN_for_XOR).ipynb',
 'logs',
 'MNIST_data',
 'Untitled.ipynb',
 'zoo.csv']

In [2]:
# XOR data setting
#tf.set_random_seed(777)  # for reproducibility
x_data = [[0, 0],
          [0, 1],
          [1, 0],
          [1, 1]]
y_data = [[0],
          [1],
          [1],
          [0]]
x_data = np.array(x_data, dtype = np.float32)
y_data = np.array(y_data, dtype = np.float32)
print(x_data, y_data)

[[ 0.  0.]
 [ 0.  1.]
 [ 1.  0.]
 [ 1.  1.]] [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]]


In [3]:
# graph setting
X = tf.placeholder(dtype = tf.float32, shape = [None, 2], name = 'x-input')
Y = tf.placeholder(dtype = tf.float32, shape = [None, 1], name = 'y-input')


with tf.name_scope('layer1') as scope:
    w1 = tf.Variable(initial_value = tf.random_normal(shape = [2,2]), name = 'weight1')
    b1 = tf.Variable(initial_value = tf.random_normal(shape = [2]), name = 'bias1')
    layer1 = tf.sigmoid(tf.matmul(X, w1) + b1)

    # from TF graph, decide which tensors you want to log
    w1_hist = tf.summary.histogram(name = 'weight1', values = w1)
    b1_hist = tf.summary.histogram(name = 'bias1', values = b1)
    layer1_hist = tf.summary.histogram(name = 'layer1', values = layer1)
    
with tf.name_scope('layer2') as scope:
    w2 = tf.Variable(initial_value = tf.random_normal(shape = [2,1]), name = 'weight2')
    b2 = tf.Variable(initial_value = tf.random_normal(shape = [1]), name = 'bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, w2) + b2)
    
    # from TF graph, decide which tensors you want to log
    w2_hist = tf.summary.histogram(name = 'weight2', values = w2)
    b2_hist = tf.summary.histogram(name = 'bias2', values = b2)
    hypothesis_hist = tf.summary.histogram(name = 'hypothesis', values = hypothesis)

# cost/loss function
with tf.name_scope('cost') as scope:
    cost = - tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis)) 
    
    # from TF graph, decide which tensors you want to log
    cost_summ = tf.summary.scalar(name = 'weight', tensor = cost)
    
with tf.name_scope('train') as scope:
    train = tf.train.GradientDescentOptimizer(learning_rate = 0.9).minimize(cost)
    
# Accuracy computation
# True if hypothesis > 0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype = tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32))
accuracy_summ = tf.summary.scalar(name = 'accuracy', tensor = accuracy)

In [4]:
# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_with_name_scope
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_with_name_scope")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(1001):
        summary, _ = sess.run([merged_summary, train], feed_dict= {X : x_data, Y : y_data})
        writer.add_summary(summary, global_step = step)

        if step % 500 == 0:
            print(step, sess.run(cost, feed_dict = {X : x_data, Y : y_data}), sess.run([w1, w2]))
    
    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X : x_data, Y : y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

0 0.709016 [array([[ 0.45735884,  0.0843783 ],
       [ 1.92094934, -1.26961207]], dtype=float32), array([[-1.05324161],
       [ 0.23504813]], dtype=float32)]
500 0.339819 [array([[-5.1176219 ,  2.42061996],
       [ 5.48095369, -2.11626196]], dtype=float32), array([[-3.95517063],
       [-2.84757304]], dtype=float32)]
1000 0.0405297 [array([[-6.75340605,  5.92167854],
       [ 6.90751934, -5.6001153 ]], dtype=float32), array([[-7.25731087],
       [-7.36639309]], dtype=float32)]

Hypothesis:  [[ 0.03581899]
 [ 0.95294172]
 [ 0.955311  ]
 [ 0.03122473]] 
Correct:  [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]] 
Accuracy:  1.0


### with tf.variable_scope

In [5]:
# jupyter notebook에서 multi graph를 실행할 때, 먼저 만들어놓은 graph는 default graph에 추가되는 형식이므로 이를 reset 해놓고
# 다시 새로운 graph를 만들어서 실행하자!
# http://stackoverflow.com/questions/35114376/error-when-computing-summaries-in-tensorflow/35117760#35117760
tf.reset_default_graph() 

# graph setting
X = tf.placeholder(dtype = tf.float32, shape = [None, 2], name = 'x-input')
Y = tf.placeholder(dtype = tf.float32, shape = [None, 1], name = 'y-input')


with tf.variable_scope('layer1'):
    w1 = tf.Variable(initial_value = tf.random_normal(shape = [2,2]), name = 'weight1')
    b1 = tf.Variable(initial_value = tf.random_normal(shape = [2]), name = 'bias1')
    layer1 = tf.sigmoid(tf.matmul(X, w1) + b1)

    # from TF graph, decide which tensors you want to log
    w1_hist = tf.summary.histogram(name = 'weight1', values = w1)
    b1_hist = tf.summary.histogram(name = 'bias1', values = b1)
    layer1_hist = tf.summary.histogram(name = 'layer1', values = layer1)
    
with tf.variable_scope('layer2'):
    w2 = tf.Variable(initial_value = tf.random_normal(shape = [2,1]), name = 'weight2')
    b2 = tf.Variable(initial_value = tf.random_normal(shape = [1]), name = 'bias2')
    hypothesis = tf.sigmoid(tf.matmul(layer1, w2) + b2)
    
    # from TF graph, decide which tensors you want to log
    w2_hist = tf.summary.histogram(name = 'weight2', values = w2)
    b2_hist = tf.summary.histogram(name = 'bias2', values = b2)
    hypothesis_hist = tf.summary.histogram(name = 'hypothesis', values = hypothesis)

# cost/loss function
with tf.variable_scope('cost'):
    cost = - tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis)) 
    
    # from TF graph, decide which tensors you want to log
    cost_summ = tf.summary.scalar(name = 'weight', tensor = cost)
    
with tf.variable_scope('train'):
    train = tf.train.GradientDescentOptimizer(learning_rate = 0.9).minimize(cost)
    
# Accuracy computation
# True if hypothesis > 0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype = tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype = tf.float32))
accuracy_summ = tf.summary.scalar(name = 'accuracy', tensor = accuracy)

In [6]:
# Launch graph
with tf.Session() as sess:
    # tensorboard --logdir=./logs/xor_with_variable_scope
    merged_summary = tf.summary.merge_all()
    writer = tf.summary.FileWriter("./logs/xor_with_variable_scope")
    writer.add_graph(sess.graph)  # Show the graph

    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in range(1001):
        summary, _ = sess.run([merged_summary, train], feed_dict= {X : x_data, Y : y_data})
        writer.add_summary(summary, global_step = step)

        if step % 500 == 0:
            print(step, sess.run(cost, feed_dict = {X : x_data, Y : y_data}), sess.run([w1, w2]))
    
    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X : x_data, Y : y_data})
    print("\nHypothesis: ", h, "\nCorrect: ", c, "\nAccuracy: ", a)

0 0.714175 [array([[-1.60497534,  1.59742951],
       [ 0.4374412 , -0.50903016]], dtype=float32), array([[ 0.49112931],
       [-1.65160561]], dtype=float32)]
500 0.0715504 [array([[-5.21596384,  6.09178448],
       [ 5.48952818, -5.85669041]], dtype=float32), array([[-6.28535652],
       [-6.23107195]], dtype=float32)]
1000 0.0263301 [array([[-6.21959352,  6.88628292],
       [ 6.49929762, -6.64041471]], dtype=float32), array([[-8.17014313],
       [-8.12390232]], dtype=float32)]

Hypothesis:  [[ 0.02334166]
 [ 0.97012681]
 [ 0.96968007]
 [ 0.02037392]] 
Correct:  [[ 0.]
 [ 1.]
 [ 1.]
 [ 0.]] 
Accuracy:  1.0
